In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from datetime import datetime

In [ ]:
sc = SparkContext(appName="DateFormatChange")

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('Employees.csv')
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
def customDateFormat(data_frame, key_field, date_field, current_dateFormat, custom_dateFormat):
    
    # converting columns into list.
    key_list = data_frame.select(key_field).rdd.flatMap(lambda x: x).collect()
    date_list = data_frame.select(date_field).rdd.flatMap(lambda x: x).collect()
    
    # Converting converting the list of dates that are in string type into date type. And after that performing date format conversion and saving it to a list
    dates = list(map(lambda x: datetime.strptime(x, current_dateFormat).strftime(custom_dateFormat), date_list))
    
    # creating a schema so that we can create a dataframe that has a key and the converted dates.
    myschema= StructType([ StructField(key_field, StringType(), True),StructField(date_field, StringType(), True)])
    key_date_df=sqlContext.createDataFrame(zip(key_list,dates),schema = myschema)
    
    # joining the original dataframe with newly created key_date data frame, and dropping the repeated key column and also the old date format column
    return data_frame.join(key_date_df, data_frame[key_field] == key_date_df[key_field],how='left').drop(data_frame[date_field]).drop(key_date_df[key_field]).select('*').sort(key_field).dropDuplicates()
     

In [ ]:

df2 = customDateFormat(df, "EMPLOYEE_ID", "HIRE_DATE", "%d-%m-%y", "%Y%m")


In [ ]:
df2.show(5)

In [ ]:
df2.printSchema()

In [ ]:
employee_id = df.select("EMPLOYEE_ID").rdd.flatMap(lambda x: x).collect()

In [ ]:
hire_date = df.select("HIRE_DATE").rdd.flatMap(lambda x: x).collect()

In [ ]:
dates = list(map(lambda x: datetime.strptime(x, "%d-%m-%y").strftime("%m/%d/%Y"), hire_date))
print(dates)

In [ ]:
myschema= StructType([ StructField("EMPLOYEE_ID", StringType(), True),StructField("HIRE_DATE", StringType(), True)])
df_date=sqlContext.createDataFrame(zip(employee_id,dates),schema = myschema)
df_date.show()

In [ ]:
dfnew = df.join(df_date, df["EMPLOYEE_ID"] == df_date["EMPLOYEE_ID"],how='full').drop(df["HIRE_DATE"]).drop(df_date["EMPLOYEE_ID"]).select('*').sort('EMPLOYEE_ID').dropDuplicates()
dfnew.show()

In [ ]:
df2 = DataFrame (dates,columns=['HIRE_DATE'])

In [ ]:
type(df2)

In [ ]:
print(df2)

In [ ]:
df=df.withColumn("HIRE_DATE",df2.HIREDATE.alias('HIRE_DATE'))
df.show(5)

In [ ]:
type(hire_date)

In [ ]:
print (hire_date)